<a href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/tutorial/skelton_back_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RustyBot(rbot)のフレームワークスケルトン(BackTest)

フレームワークで動かすためのBotの構造サンプルを提供します。

`on_init`, `on_tick`, `on_clock`, `on_update`を必要に応じて実装します。

バックテストはColabで動きます。試してみてください。

参考：
forward, real_runはWebSocketが繋げるローカル環境が必要です。
* [forward test](./binance_skelton_forward.ipynb)
* [real run](./binance_skelton_realrun.ipynb)

## バックテストの準備

In [ ]:
# rbotをインストールします。
! pip install --upgrade pip
! pip install rbot

In [ ]:

from rbot import time_string

class SkeltonAgent:      # クラス名は任意です
    def __init__(self):
        """Botの初期化処理です。パラメータなどを設定するといいでしょう。利用しなくても構いません。
        """
        self.tick_count = 0         # on_tickが呼び出された回数をカウントします。

    def on_init(self, session):
        """Botの初期化処理。Botの初期化時に一度だけ呼ばれます。
        通常はsession.clock_interval_secを指定しon_clockの呼び出し間隔を設定します。
        Args:
            session: セッション情報（Botの初期化時用に渡されます）
        """
        session.clock_interval_sec = 60 * 60 * 1        # 1時間ごとにon_clockを呼び出す


    def on_tick(self, session, side, price, size):
        """取引所からの全ての約定イベント毎に呼び出される処理です（高頻度で呼び出されます）
        Args:
            session: セッション情報（市況情報の取得や注文するために利用します)
            side: 売買区分です。"Buy"または"Sell"が設定されます。
            price: 約定価格です。
            size: 約定数量です。
        """

        # on_tickは高頻度によびだされるので、１万回に1回だけ内容をプリントします。
        if self.tick_count % 10_000 == 0:
            print("on_tick: ", side, price, size)

        self.tick_count += 1

    def on_clock(self, session, clock):
        """定期的にフレームワークから呼び出される処理です。
        session.clock_interval_secで指定した間隔で呼び出されます。

        Args:
            session: セッション情報（市況情報の取得や注文するために利用します)
            clock: 現在時刻です。エポック時間からのマイクロ秒で表されます。
        """
        # 現在の時刻をプリントします。
        print("on_clock: ", clock, ": ", time_string(clock))

    def on_update(self, session, updated_order):
        """自分の注文状態が変化した場合に呼び出される処理です。
        Args:
            session: セッション情報（市況情報の取得や注文するために利用します)
            session: セッション情報（市況情報の取得や注文するために利用します)
            updated_order: 注文状態が変化した注文情報です。
        """
        # 注文状態が変化した注文情報をプリントします。オーダーを発行しない限り呼び出されません。
        print("on_update", updated_order)



In [ ]:
# Google Colabの場合は、GoogleDriveにDBをつくります。
if 'google.colab' in str(get_ipython()):
    # Google Driveをマウントする
    from google.colab import drive
    drive.mount('/content/drive')

    # DBの保存先をGoogle Driveに変更する
    import os
    os.environ['RBOT_DB_ROOT'] = '/content/drive/MyDrive'    # MyDrive以下を指定しましたが適宜変更ください


In [ ]:
# Binanceマーケットを指定します。(BinanceかBybitのどちらかを選択してください。)
from rbot import Binance
from rbot import BinanceConfig


binance_exchange = Binance(production=True)     # 本番ネットのデータを取得します。

config = BinanceConfig.BTCUSDT          # BTC/USDTの市況情報を取得します。
binance_market = binance_exchange.open_market(config)   # BTCUSDTの市況情報を取得するためのマーケットを開きます。


In [ ]:
# Bybitマーケットを指定します。(binanceかbybitのどちらか一方を選択してください)
from rbot import Bybit
from rbot import BybitConfig



bybit_exchange = Bybit(production=True)     # 本番ネットのデータを取得します。

config = BybitConfig.BTCUSDT          # BTC/USDTの市況情報を取得します。
bybit_market = bybit_exchange.open_market(config)   # BTCUSDTの市況情報を取得するためのマーケットを開きます。


In [ ]:
#exchange = bybit_exchange
#market = bybit_market
exchange = binance_exchange
market = binance_market

In [ ]:
#  アーカイブ以外、WSやRESTからのデータを削除します
market.expire_unfix_data(force=True)        

In [ ]:
# 過去ログを１日分ダウンロードします。
# また大量のファイルをダウンロードするため、ディスク容量に注意してください。
#  環境変数 RBOT_DB_ROOTで保存場所を変更できます。
market.download_archive(ndays=1, verbose=True)

In [ ]:
# DBの場所は以下の場所にあります。ディスクを解放する場合や、古いバージョンのrbotと非互換が発生した場合などDBファイルを一旦削除してください。
market.file_name

In [ ]:
from rbot import Runner
from rbot import NOW, DAYS

agent = SkeltonAgent()
runner = Runner()

session = runner.back_test(
                exchange=exchange,
                market=market,
                agent=agent,
                start_time=0,
                end_time=0,
                verbose=True
            )

session

In [ ]:
# 実行したセッションからログオブジェクトを取得して分析します。
# 今回はAgentがなにもしていないのでログがありませんが、
# 以下流れを示します。
log = session.log

In [ ]:
# 今回はオーダー発行していないので出力されません。
log.orders

In [ ]:
# 今回はインジケーターを保存していないので出力されません。
# Agent中で session.log_indicator("indicator", 1.0)
# のように保存すると、保存時刻と値がDataFrameで取得できます。
# キーとなる文字列は任意のものが任意数利用できます。
log['indicators']